### training scheme
#### (1) train denoising auto encoder model using all data including train and test data
#### (2) from the weights of denoising auto encoder model, finetune to predict targets such as reactivity

### rough network architecture
#### inputs -> conv1ds -> aggregation of neighborhoods -> multi head attention -> aggregation of neighborhoods -> multi head attention -> conv1d -> predict
#### this architecture was inspired by https://www.kaggle.com/cpmpml/graph-transfomer


In [1]:
pretrain_dir = None # model dir for resuming training. if None, train from scrach

one_fold = False # if True, train model at only first fold. use if you try a new idea quickly.
run_test = False # if True, use small data. you can check whether this code run or not
denoise = True # if True, use train data whose signal_to_noise > 1

ae_epochs = 20 # epoch of training of denoising auto encoder
ae_epochs_each = 5 # epoch of training of denoising auto encoder each time. 
                   # I use train data (seqlen = 107) and private test data (seqlen = 130) for auto encoder training.
                   # I dont know how to easily fit keras model to use both of different shape data simultaneously, 
                   # so I call fit function several times. 
ae_batch_size = 32

epochs_list = [30, 10, 3, 3, 5, 5]
batch_size_list = [8, 16, 32, 64, 128, 256] 

## copy pretrain model to working dir
import shutil
import glob
if pretrain_dir is not None:
    for d in glob.glob(pretrain_dir + "*"):
        shutil.copy(d, ".")
    
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import os
import matplotlib.pyplot as plt
%matplotlib inline

## load

In [2]:
import json
import glob
from tqdm.notebook import tqdm

data = pd.read_json("../data/train.json",lines=True)
if denoise:
    data = data[data.signal_to_noise > 1].reset_index(drop = True)

## target

In [3]:
targets = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

data_labels = []
seq_len = data["seq_length"].iloc[0]
seq_len_target = data["seq_scored"].iloc[0]
ignore = -10000
ignore_length = seq_len - seq_len_target
for target in targets:
    y = np.vstack(data[target])
    dummy = np.zeros([y.shape[0], ignore_length]) + ignore
    y = np.hstack([y, dummy])
    data_labels.append(y)
data_labels = np.stack(data_labels, axis = 2)
data_labels.shape

(2096, 107, 5)

In [4]:
data.shape

(2096, 19)

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    data, data_labels, test_size=.1, random_state=47, 
    stratify=data.SN_filter
)

In [6]:
if run_test: ## to test 
    data = data[:30]

As = []
for id in tqdm(x_train["id"]):
    a = np.load(f"../data/bpps/{id}.npy")
    As.append(a)
As = np.array(As)
As_pub = []
for id in tqdm(x_test["id"]):
    a = np.load(f"../data/bpps/{id}.npy")
    As_pub.append(a)
As_pub = np.array(As_pub)

In [7]:
print(x_train.shape)
x_train.head()

(1886, 19)


,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
1452,1661,id_b4aa34fee,GGAAAGCAGCGCCGGAAGGCACAGCCGCAAAACUAAAGGCGAAAAA...,........(.(((....))).).(((((...(((...((((........,EEEEEEEESISSSHHHHSSSISMSSSSSIIISSSIIISSSSHHHHH...,3.974,1,107,68,"[0.1917, 0.24550000000000002, 0.22060000000000...","[0.25730000000000003, 0.2838, 0.21030000000000...","[0.324, 0.3235, 0.21960000000000002, 0.158, 0....","[0.18130000000000002, 0.2024, 0.18630000000000...","[0.2751, 0.2983, 0.2523, 0.1961, 0.2083, 0.202...","[0.5057, 1.7747000000000002, 1.5566, 0.906, 1....","[0.9389000000000001, 1.7486000000000002, 0.941...","[1.9460000000000002, 3.1451000000000002, 1.271...","[0.3917, 1.1639, 1.0394, 0.5305, 0.67270000000...","[0.6966, 1.8341, 1.3003, 0.7393000000000001, 0..."
1705,1951,id_d19174185,GGAAAUAUAAAUUUCAGCUUCACAUUCCUUUCGCAUUCGAAAGAAU...,............(((..((((.((((((((((.(((((....))))...,EEEEEEEEEEEESSSIISSSSBSSSSSSSSSSISSSSSHHHHSSSS...,3.317,1,107,68,"[0.2637, 0.35350000000000004, 0.24980000000000...","[0.44570000000000004, 0.5985, 0.3523, 0.3275, ...","[0.5332, 0.6131, 0.3215, 0.322, 0.3284, 0.3784...","[0.21150000000000002, 0.3528, 0.2285, 0.2427, ...","[0.39640000000000003, 0.517, 0.3624, 0.2995, 0...","[0.7349, 1.6002, 0.9375, 0.7302000000000001, 1...","[0.8748, 2.5291, 0.7826000000000001, 0.7193, 2...","[2.7114000000000003, 3.5916, 0.8818, 1.0267, 1...","[0.343, 1.6571, 0.7803, 1.0464, 1.3912, 1.5941...","[1.0008, 1.7982, 1.1282, 0.7133, 1.74460000000..."
242,273,id_1bd260b05,GGAAACGCAGCAGGCAGAUGACGCGGAAAACAUAGGAAAAACUAUG...,.....(((.((.(.((.(((.((((.....(((((......)))))...,EEEEESSSBSSBSISSISSSBSSSSIIIIISSSSSHHHHHHSSSSS...,4.740,1,107,68,"[0.2053, 0.25, 0.22080000000000002, 0.176, 0.1...","[0.28850000000000003, 0.3342, 0.2634, 0.204500...","[0.3164, 0.3764, 0.2298, 0.19690000000000002, ...","[0.1837, 0.2802, 0.19720000000000001, 0.1958, ...","[0.2155, 0.366, 0.29610000000000003, 0.2258, 0...","[0.8514, 1.3002, 1.4004, 0.9162, 0.1922, 0.104...","[0.7996000000000001, 1.2032, 0.874400000000000...","[1.9604, 2.5952, 1.0739, 0.877, 0.3819, 0.6940...","[0.501, 1.5292, 0.8887, 1.0166, 0.470400000000...","[0.3069, 1.5297, 1.4404, 0.8256, 0.2597, 0.503..."
1303,1492,id_a0fc45be8,GGAAAGCGCACGAGGCCGAUCGAGAAUGGAUUGCAGCGAGAGAGGC...,......(((.((.((((..((..(..((.....)).)..))..)))...,EEEEEESSSISSISSSSIISSIISIISSHHHHHSSISIISSIISSS...,7.068,1,107,68,"[0.14880000000000002, 0.1713, 0.1631, 0.1199, ...","[0.1993, 0.2436, 0.2044, 0.1346, 0.1024, 0.098...","[0.233, 0.2389, 0.16390000000000002, 0.1149, 0...","[0.14850000000000002, 0.2366, 0.1621, 0.108400...","[0.2056, 0.2429, 0.1832, 0.1279, 0.1254, 0.093...","[0.7467, 1.4459, 1.5955, 0.8522000000000001, 0...","[0.8696, 1.9521000000000002, 1.596, 0.6469, 0....","[2.1264, 2.9469000000000003, 1.4311, 0.6401, 0...","[0.5842, 3.1236, 1.5319, 0.6339, 0.4465, 0.237...","[1.0084, 2.2792, 1.3965, 0.6178, 0.6412, 0.285..."
764,869,id_5a917f313,GGAAACGUACGGUACGGUUGUCGGUUAGGCAACUGCUGUCUGCAGA...,...............(.((((.((.((((((..(((((((....))...,EEEEEEEEEEEEEEESISSSSISSISSSSSSBBSSSSSSSHHHHSS...,1.891,1,107,68,"[0.363, 0.5069, 0.3811, 0.4199, 0.287000000000...","[0.5436, 0.7083, 0.6357, 0.42310000000000003, ...","[0.6141, 0.6917, 0.4786, 0.4057, 0.2313, 0.401...","[0.2614, 0.4813, 0.4828, 0.3547, 0.2636, 0.284...","[0.5109, 0.6109, 0.48660000000000003, 0.4445, ...","[0.337, 1.3322, 0.8707, 1.207, 0.4961000000000...","[0.4277, 1.3516, 1.4038, 0.4153, 0.0, 0.3008, ...","[1.0879, 1.6128, 0.9981000000000001, 0.4722, 0...","[-0.1558, 0.7852, 1.4011, 0.5535, 0.2865000000...","[0.4645, 0.9849, 1.0028, 0.6905, 0.34340000000..."


## structure adj

In [8]:
def get_structure_adj(train):
    ## get adjacent matrix from structure sequence
    
    ## here I calculate adjacent matrix of each base pair, 
    ## but eventually ignore difference of base pair and integrate into one matrix
    Ss = []
    for i in tqdm(range(len(train))):
        seq_length = train["seq_length"].iloc[i]
        structure = train["structure"].iloc[i]
        sequence = train["sequence"].iloc[i]

        cue = []
        a_structures = {
            ("A", "U") : np.zeros([seq_length, seq_length]),
            ("C", "G") : np.zeros([seq_length, seq_length]),
            ("U", "G") : np.zeros([seq_length, seq_length]),
            ("U", "A") : np.zeros([seq_length, seq_length]),
            ("G", "C") : np.zeros([seq_length, seq_length]),
            ("G", "U") : np.zeros([seq_length, seq_length]),
        }
        a_structure = np.zeros([seq_length, seq_length])
        for i in range(seq_length):
            if structure[i] == "(":
                cue.append(i)
            elif structure[i] == ")":
                start = cue.pop()
#                 a_structure[start, i] = 1
#                 a_structure[i, start] = 1
                a_structures[(sequence[start], sequence[i])][start, i] = 1
                a_structures[(sequence[i], sequence[start])][i, start] = 1
        
        a_strc = np.stack([a for a in a_structures.values()], axis = 2)
        a_strc = np.sum(a_strc, axis = 2, keepdims = True)
        Ss.append(a_strc)
    
    Ss = np.array(Ss)
    print(Ss.shape)
    return Ss
Ss = get_structure_adj(x_train)
Ss_pub = get_structure_adj(x_test)


(1886, 107, 107, 1)



(210, 107, 107, 1)


## distance adj

In [9]:
def get_distance_matrix(As):
    ## adjacent matrix based on distance on the sequence
    ## D[i, j] = 1 / (abs(i - j) + 1) ** pow, pow = 1, 2, 4
    
    idx = np.arange(As.shape[1])
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1/Ds
    Ds = Ds[None, :,:]
    Ds = np.repeat(Ds, len(As), axis = 0)
    
    Dss = []
    for i in [1, 2, 4]: 
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis = 3)
    print(Ds.shape)
    return Ds

Ds = get_distance_matrix(As)
Ds_pub = get_distance_matrix(As_pub)

(1886, 107, 107, 3)
(210, 107, 107, 3)


In [10]:
## concat adjecent
As = np.concatenate([As[:,:,:,None], Ss, Ds], axis = 3).astype(np.float32)
As_pub = np.concatenate([As_pub[:,:,:,None], Ss_pub, Ds_pub], axis = 3).astype(np.float32)
del Ss, Ds, Ss_pub, Ds_pub
As.shape, As_pub.shape

((1886, 107, 107, 5), (210, 107, 107, 5))

## node

In [11]:
## sequence
def return_ohe(n, i):
    tmp = [0] * n
    tmp[i] = 1
    return tmp

def get_input(train):
    ## get node features, which is one hot encoded
    mapping = {}
    vocab = ["A", "G", "C", "U"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_node = np.stack(train["sequence"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))

    mapping = {}
    vocab = ["S", "M", "I", "B", "H", "E", "X"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_loop = np.stack(train["predicted_loop_type"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    mapping = {}
    vocab = [".", "(", ")"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_structure = np.stack(train["structure"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    
    X_node = np.concatenate([X_node, X_loop], axis = 2)
    
    ## interaction
    a = np.sum(X_node * (2 ** np.arange(X_node.shape[2])[None, None, :]), axis = 2)
    vocab = sorted(set(a.flatten()))
    print(vocab)
    ohes = []
    for v in vocab:
        ohes.append(a == v)
    ohes = np.stack(ohes, axis = 2)
    X_node = np.concatenate([X_node, ohes], axis = 2).astype(np.float32)
    
    
    print(X_node.shape)
    return X_node

X_node = get_input(x_train)
X_node_pub = get_input(x_test)

[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(1886, 107, 39)
[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(210, 107, 39)


## model

In [12]:
import tensorflow as tf
from tensorflow.keras import layers as L
import tensorflow_addons as tfa
from tensorflow.keras import backend as K

def mcrmse(t, p, seq_len_target = seq_len_target):
    ## calculate mcrmse score by using numpy
    t = t[:, :seq_len_target]
    p = p[:, :seq_len_target]
    
    score = np.mean(np.sqrt(np.mean(np.mean((p - t) ** 2, axis = 1), axis = 0)))
    return score

def mcrmse_loss(t, y, seq_len_target = seq_len_target):
    ## calculate mcrmse score by using tf
    t = t[:, :seq_len_target]
    y = y[:, :seq_len_target]
    
    loss = tf.reduce_mean(tf.sqrt(tf.reduce_mean(tf.reduce_mean((t - y) ** 2, axis = 1), axis = 0)))
    return loss

def attention(x_inner, x_outer, n_factor, dropout):
    x_Q =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_inner)
    x_K =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_V =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_KT = L.Permute((2, 1))(x_K)
    res = L.Lambda(lambda c: K.batch_dot(c[0], c[1]) / np.sqrt(n_factor))([x_Q, x_KT])
#     res = tf.expand_dims(res, axis = 3)
#     res = L.Conv2D(16, 3, 1, padding = "same", activation = "relu")(res)
#     res = L.Conv2D(1, 3, 1, padding = "same", activation = "relu")(res)
#     res = tf.squeeze(res, axis = 3)
    att = L.Lambda(lambda c: K.softmax(c, axis=-1))(res)
    att = L.Lambda(lambda c: K.batch_dot(c[0], c[1]))([att, x_V])
    return att

def multi_head_attention(x, y, n_factor, n_head, dropout):
    if n_head == 1:
        att = attention(x, y, n_factor, dropout)
    else:
        n_factor_head = n_factor // n_head
        heads = [attention(x, y, n_factor_head, dropout) for i in range(n_head)]
        att = L.Concatenate()(heads)
        att = L.Dense(n_factor, 
                      kernel_initializer='glorot_uniform',
                      bias_initializer='glorot_uniform',
                     )(att)
    x = L.Add()([x, att])
    x = L.LayerNormalization()(x)
    if dropout > 0:
        x = L.Dropout(dropout)(x)
    return x

def res(x, unit, kernel = 3, rate = 0.1):
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.LeakyReLU()(h)
    h = L.Dropout(rate)(h)
    return L.Add()([x, h])

def forward(x, unit, kernel = 3, rate = 0.1):
#     h = L.Dense(unit, None)(x)
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.Dropout(rate)(h)
#         h = tf.keras.activations.swish(h)
    h = L.LeakyReLU()(h)
    h = res(h, unit, kernel, rate)
    return h

def adj_attn(x, adj, unit, n = 2, rate = 0.1):
    x_a = x
    x_as = []
    for i in range(n):
        x_a = forward(x_a, unit)
        x_a = tf.matmul(adj, x_a) ## aggregate neighborhoods
        x_as.append(x_a)
    if n == 1:
        x_a = x_as[0]
    else:
        x_a = L.Concatenate()(x_as)
    x_a = forward(x_a, unit)
    return x_a


def get_base(config):
    ## base model architecture 
    ## node, adj -> middle feature
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")
    
    adj_learned = L.Dense(1, "relu")(adj)
    adj_all = L.Concatenate(axis = 3)([adj, adj_learned])
        
    xs = []
    xs.append(node)
    x1 = forward(node, 128, kernel = 3, rate = 0.0)
    x2 = forward(x1, 64, kernel = 6, rate = 0.0)
    x3 = forward(x2, 32, kernel = 15, rate = 0.0)
    x4 = forward(x3, 16, kernel = 30, rate = 0.0)
    x = L.Concatenate()([x1, x2, x3, x4])
    
    for unit in [64, 32]:
        x_as = []
        for i in range(adj_all.shape[3]):
            x_a = adj_attn(x, adj_all[:, :, :, i], unit, rate = 0.0)
            x_as.append(x_a)
        x_c = forward(x, unit, kernel = 30)
        
        x = L.Concatenate()(x_as + [x_c])
        x = forward(x, unit)
        x = multi_head_attention(x, x, unit, 4, 0.0)
        xs.append(x)
        
    x = L.Concatenate()(xs)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    return model


def get_ae_model(base, config):
    ## denoising auto encoder part
    ## node, adj -> middle feature -> node
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")

    x = base([L.SpatialDropout1D(0.3)(node), adj])
    x = forward(x, 64, rate = 0.3)
    p = L.Dense(X_node.shape[2], "sigmoid")(x)
    
    loss = - tf.reduce_mean(20 * node * tf.math.log(p + 1e-4) + (1 - node) * tf.math.log(1 - p + 1e-4))
    model = tf.keras.Model(inputs = [node, adj], outputs = [loss])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = lambda t, y : y)
    return model


def get_model(base, config):
    ## regression part
    ## node, adj -> middle feature -> prediction of targets
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")
    
    x = base([node, adj])
    x = forward(x, 128, rate = 0.4)
    x = L.Dense(5, None)(x)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = mcrmse_loss)
    return model

def get_optimizer():
#     sgd = tf.keras.optimizers.SGD(0.05, momentum = 0.9, nesterov=True)
    adam = tf.optimizers.Adam()
#     radam = tfa.optimizers.RectifiedAdam()
#     lookahead = tfa.optimizers.Lookahead(adam, sync_period=6)
#     swa = tfa.optimizers.SWA(adam)
    return adam

## pretrain

In [ ]:
## here train denoising auto encoder model using all data

config = {} ## not use now
if ae_epochs > 0:
    base = get_base(config)
    ae_model = get_ae_model(base, config)
    ## TODO : simultaneous train
    for i in range(ae_epochs//ae_epochs_each):
        print(f"------ {i} ------")
        print("--- train ---")
        ae_model.fit([X_node, As], [X_node[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        print("--- public ---")
        ae_model.fit([X_node_pub, As_pub], [X_node_pub[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        print("--- private ---")
        ae_model.fit([X_node_pri, As_pri], [X_node_pri[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        gc.collect()
    print("****** save ae model ******")
    base.save_weights("./base_ae")

## train

In [ ]:
## here train regression model from pretrain auto encoder model

from sklearn.model_selection import KFold
kfold = KFold(5, shuffle = True, random_state = 42)

scores = []
preds = np.zeros([len(X_node), X_node.shape[1], 5])
for i, (tr_idx, va_idx) in enumerate(kfold.split(X_node, As)):
    print(f"------ fold {i} start -----")
    print(f"------ fold {i} start -----")
    print(f"------ fold {i} start -----")
    X_node_tr = X_node[tr_idx]
    X_node_va = X_node[va_idx]
    As_tr = As[tr_idx]
    As_va = As[va_idx]
    y_tr = y[tr_idx]
    y_va = y[va_idx]
    
    base = get_base(config)
    if ae_epochs > 0:
        print("****** load ae model ******")
        base.load_weights("./base_ae")
    model = get_model(base, config)
    if pretrain_dir is not None:
        d = f"./model{i}"
        print(f"--- load from {d} ---")
        model.load_weights(d)
    for epochs, batch_size in zip(epochs_list, batch_size_list):
        print(f"epochs : {epochs}, batch_size : {batch_size}")
        model.fit([X_node_tr, As_tr], [y_tr],
                  validation_data=([X_node_va, As_va], [y_va]),
                  epochs = epochs,
                  batch_size = batch_size, validation_freq = 3)
        
    model.save_weights(f"./model{i}")
    p = model.predict([X_node_va, As_va])
    scores.append(mcrmse(y_va, p))
    print(f"fold {i}: mcrmse {scores[-1]}")
    preds[va_idx] = p
    if one_fold:
        break
        
pd.to_pickle(preds, "oof.pkl")

In [ ]:
print(scores)

## predict

In [ ]:
p_pub = 0
p_pri = 0
for i in range(5):
    model.load_weights(f"./model{i}")
    p_pub += model.predict([X_node_pub, As_pub]) / 5
    p_pri += model.predict([X_node_pri, As_pri]) / 5
    if one_fold:
        p_pub *= 5
        p_pri *= 5
        break

for i, target in enumerate(targets):
    test_pub[target] = [list(p_pub[k, :, i]) for k in range(p_pub.shape[0])]
    test_pri[target] = [list(p_pri[k, :, i]) for k in range(p_pri.shape[0])]

## sub

In [ ]:
preds_ls = []
for df, preds in [(test_pub, p_pub), (test_pri, p_pri)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=targets)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.to_csv("submission.csv", index = False)
preds_df.head()

In [ ]:
print(scores)
print(np.mean(scores))